## Installs & Imports

In [0]:
# Select Tensorflow 2.x version in Colab
%tensorflow_version 2.x

# Import TensorFlow and tf.keras
import tensorflow as tf
keras = tf.keras

# Import helper libraries
import numpy as np
import matplotlib.pyplot as plt

# Print TensorFlow version
version = tf.__version__
print(version)

## Data


 ### 1. Load the MNIST dataset

In [0]:
# Load mnist from keras datasets
mnist = keras.datasets.mnist

In [0]:
# Get the training and test data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. Explore the MNIST dataset

In [0]:
# Inspect the training and test dataset shape
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, "y_test shape", y_test.shape)

In [0]:
# Take a look at one of the training images
index = 0

In [0]:
# First let's look at the label of the image
print(y_train[index])

In [0]:
plt.imshow(x_train[index], cmap="gray")

In [0]:
print(x_train[index])

In [0]:
# Check datatype of x_train
x_train.dtype

### 2. Data preprocessing

In [0]:
# Convert data to float32 and normalize the input data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [0]:
x_train.dtype

In [0]:
x_train[index]

In [0]:
# Reshape input data from (28, 28) to (28, 28, 1)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [0]:
# Take a look at the dataset shape after conversion with keras.utils.to_categorical
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, "y_test shape", y_test.shape)

In [0]:
# One-hot encode the labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [0]:
y_train[index]

## Model Training

### Define the model architecture
There are 3 ways to define a model with tf.Keras:
1. Sequential API
2. Functional API
3. Model subclassing

We will create a simple Convolutional Neural Network with the Sequential model API.

![alt text](https://drive.google.com/uc?id=1Ci3xJ7zJ1rhAB_b_sHWwjrPAjl-8gCbY)

In [0]:
def create_model():
  
  # Define the model architecture
  model = keras.models.Sequential([
    # Must define the input shape in the first layer of the neural network
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=2),

    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),

    keras.layers.Flatten(),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  # Compile the model
  model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),
         metrics=['accuracy'])
      
  return model

In [0]:
model = create_model()

# Take a look at the model summary
model.summary()

In [0]:
%%time 
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=3)

### Model evaluation

In [0]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print("Test Loss", test_loss)
print('Test Accuracy:', test_accuracy)

In [0]:
predictions = model.predict(x_test)

In [0]:
index = 99

In [0]:
np.argmax(predictions[index])

In [0]:
plt.imshow(np.squeeze(x_test[index]))

## Model Conversion/Inference with TFLite

We will use TensorFlow Lite (TFLite) tools for model conversion and inference. The **Converter** will be used for conversion and the **Interpreter** will be used for inference. 

![alt text](https://drive.google.com/uc?id=1XXk8ItBcy1W3k4KMw6so5dZR-y4YNNzk)

Follow these steps:
*   Convert to a TFLite model (with the Converter)
*   Validate the TFLite model (with the Interpreter)
*   Save and download the TFLite Model





### Convert tf.Keras model to TFLite 

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) 

In [0]:
tflite_model = converter.convert()

### Evaluate the TFLite model
After converting the tf.Keras model to tflite format, it’s important to validate that it is performing on par as your original tf.Keras model. 


First we get a test image from the test dataset

In [0]:
# You can get any test image
index = 8
test_image = x_test[index]
test_image.shape

In [0]:
# Add batch dimension since a tensor expects the shape of [batch, image height, image width, color channel]
test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
test_image.shape

Below are some boilerplate low-level TensorFlow code for running the interpreter

In [0]:
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensor index
input_tensor_index = interpreter.get_input_details()[0]['index']
output_tensor_index = interpreter.get_output_details()[0]['index']

# Set the value of the input tensor as the test image
interpreter.set_tensor(input_tensor_index, test_image)

# Run inference with the interpreter
interpreter.invoke()

Now let's compare the tflite model prediction with the original tf.Keras model prediction:

In [0]:
# Here is the digit that the tflite model predicts
tflite_prediction = interpreter.get_tensor(output_tensor_index)
np.argmax(tflite_prediction)

In [0]:
# Here is the digit that the tf.Keras model predicts
model_prediction = model(tf.constant(test_image))
np.argmax(model_prediction)

### Save and download the TFLite model

First let's save the TFLite model to a file:

In [0]:
tflite_model_file_name = "mnist.tflite"
# Open a file for writing in binary mode
file = open(tflite_model_file_name, 'wb')
# Write to the file
file.write(tflite_model)
# Close the file afterwards
file.close()

Download the tflite model:

In [0]:
# Take a look at the files in the directory
import os
os.listdir(".")

In [0]:
# Download the .tflite model file for deployment to Android
from google.colab import files
files.download(tflite_model_file_name)